In [ ]:
# | default_exp _components.asyncapi

In [ ]:
# | export

import json
import platform
import shutil
import subprocess  # nosec: B404: Consider possible security implications associated with the subprocess module.
import tempfile
from datetime import timedelta
from enum import Enum
from pathlib import Path
from typing import *

from pydantic import BaseModel, Field, HttpUrl
from pydantic.json import timedelta_isoformat
from pydantic.schema import schema

from fastkafka._components.aiokafka_consumer_loop import ConsumeCallable
from fastkafka._components.docs_dependencies import _check_npm_with_local
from fastkafka._components.helpers import unwrap_list_type
from fastkafka._components.logger import get_logger
from fastkafka._components.producer_decorator import (
    ProduceCallable,
    unwrap_from_kafka_event,
)

In [ ]:
from datetime import datetime

import pytest
from pydantic import EmailStr
from rich.pretty import pprint

from fastkafka._components.aiokafka_consumer_loop import EventMetadata
from fastkafka._components.logger import suppress_timestamps
from fastkafka._components.producer_decorator import KafkaEvent

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export


class KafkaMessage(BaseModel):
    class Config:
        """This class is used for specific JSON encoders, in our case to properly format timedelta in ISO format."""

        json_encoders = {
            timedelta: timedelta_isoformat,
        }

In [ ]:
# | output: false


class MyMsg(KafkaMessage):
    dt: datetime = Field(..., example=datetime.now())
    td: timedelta = Field(timedelta(days=1, hours=12, minutes=2, seconds=1.2345678))


my_msg = MyMsg(dt=datetime(year=2021, month=12, day=31, hour=23, minute=59, second=58))
pprint(my_msg)
expected = '{"dt": "2021-12-31T23:59:58", "td": "P1DT12H2M1.234568S"}'
actual = my_msg.json()
assert actual == expected, f"{actual} != {expected}"

actual = MyMsg.parse_raw(actual)
assert actual == my_msg

MyMsg(
│   dt=datetime.datetime(2021, 12, 31, 23, 59, 58),
│   td=datetime.timedelta(days=1, seconds=43321, microseconds=234568)
)

In [ ]:
# | export


class SecurityType(str, Enum):
    plain = "plain"
    userPassword = "userPassword"
    apiKey = "apiKey"
    X509 = "X509"
    symmetricEncryption = "symmetricEncryption"
    asymmetricEncryption = "asymmetricEncryption"
    httpApiKey = "httpApiKey"
    http = "http"
    oauth2 = "oauth2"
    openIdConnect = "openIdConnect"
    scramSha256 = "scramSha256"
    scramSha512 = "scramSha512"
    gssapi = "gssapi"


class APIKeyLocation(str, Enum):
    user = "user"
    password = "password"  # nosec
    query = "query"
    header = "header"
    cookie = "cookie"


sec_scheme_name_mapping = {"security_type": "type", "api_key_loc": "in"}


class SecuritySchema(BaseModel):
    security_type: SecurityType = Field(..., example="plain")
    description: Optional[str] = Field(None, example="My security scheme")
    name: Optional[str] = Field(None, example="my_secret_scheme")
    api_key_loc: Optional[APIKeyLocation] = Field(None, example="user")
    scheme: Optional[str] = None
    bearerFormat: Optional[str] = None
    flows: Optional[str] = None
    openIdConnectUrl: Optional[str] = None

    def __init__(self, **kwargs: Any):
        for k, v in sec_scheme_name_mapping.items():
            if v in kwargs:
                kwargs[k] = kwargs.pop(v)
        super().__init__(**kwargs)

    def dict(self, *args: Any, **kwarg: Any) -> Dict[str, Any]:
        """Renames internal names of members ('security_type' -> 'type', 'api_key_loc' -> 'in')"""
        d = super().dict(*args, **kwarg)

        for k, v in sec_scheme_name_mapping.items():
            d[v] = d.pop(k)

        # removes None values
        d = {k: v for k, v in d.items() if v is not None}

        return d

    def json(self, *args: Any, **kwargs: Any) -> str:
        """Serialize into JSON using dict()"""
        return json.dumps(self.dict(), *args, **kwargs)

In [ ]:
# | output: false

sec_schema = SecuritySchema(type="plain")
pprint(sec_schema)

actual = sec_schema.json()
print(f"JSON={actual}")
assert actual == '{"type": "plain"}', actual

actual = SecuritySchema.parse_raw(sec_schema.json())
pprint(actual)
assert actual == sec_schema

SecuritySchema(
│   security_type=<SecurityType.plain: 'plain'>,
│   description=None,
│   name=None,
│   api_key_loc=None,
│   scheme=None,
│   bearerFormat=None,
│   flows=None,
│   openIdConnectUrl=None
)

JSON={"type": "plain"}


SecuritySchema(
│   security_type=<SecurityType.plain: 'plain'>,
│   description=None,
│   name=None,
│   api_key_loc=None,
│   scheme=None,
│   bearerFormat=None,
│   flows=None,
│   openIdConnectUrl=None
)

In [ ]:
# | export


class KafkaBroker(BaseModel):
    """Kafka broker"""

    url: str = Field(..., example="localhost")
    description: str = Field("Kafka broker")
    port: str = Field("9092")
    protocol: str = Field("kafka")
    security: Optional[SecuritySchema] = None

    def dict(self, *args: Any, **kwarg: Any) -> Dict[str, Any]:
        """Makes port a variable and remove it from the dictionary"""
        d = super().dict(*args, **kwarg)
        d["variables"] = {"port": {"default": self.port}}
        d.pop("port")

        d = {k: v for k, v in d.items() if v is not None}

        return d

    def json(self, *args: Any, **kwargs: Any) -> str:
        """Serialize into JSON using dict()"""
        return json.dumps(self.dict(), *args, **kwargs)

In [ ]:
# | output: false

kafka_broker = KafkaBroker(url="kafka")
pprint(kafka_broker)

expected = '{"url": "kafka", "description": "Kafka broker", "protocol": "kafka", "variables": {"port": {"default": "9092"}}}'
print(kafka_broker.json())
assert kafka_broker.json() == expected

# serialization/deserialization test
actual = KafkaBroker.parse_raw(kafka_broker.json())
assert actual == kafka_broker

KafkaBroker(url='kafka', description='Kafka broker', port='9092', protocol='kafka', security=None)

{"url": "kafka", "description": "Kafka broker", "protocol": "kafka", "variables": {"port": {"default": "9092"}}}


In [ ]:
# | output: false

sec_kafka_broker = KafkaBroker(
    url="kafka", protocol="kafka-secure", security=SecuritySchema(type="plain")
)
pprint(sec_kafka_broker)

expected = '{"url": "kafka", "description": "Kafka broker", "protocol": "kafka-secure", "security": {"type": "plain"}, "variables": {"port": {"default": "9092"}}}'
actual = sec_kafka_broker.json()
print(f"JSON={actual}")
assert actual == expected

# serialization/deserialization test
actual = KafkaBroker.parse_raw(sec_kafka_broker.json())
assert actual == sec_kafka_broker

KafkaBroker(
│   url='kafka',
│   description='Kafka broker',
│   port='9092',
│   protocol='kafka-secure',
│   security=SecuritySchema(
│   │   security_type=<SecurityType.plain: 'plain'>,
│   │   description=None,
│   │   name=None,
│   │   api_key_loc=None,
│   │   scheme=None,
│   │   bearerFormat=None,
│   │   flows=None,
│   │   openIdConnectUrl=None
│   )
)

JSON={"url": "kafka", "description": "Kafka broker", "protocol": "kafka-secure", "security": {"type": "plain"}, "variables": {"port": {"default": "9092"}}}


In [ ]:
# | export


class ContactInfo(BaseModel):
    name: str = Field(..., example="My company")
    url: HttpUrl = Field(..., example="https://www.github.com/mycompany")
    email: str = Field(..., example="noreply@mycompany.com")


class KafkaServiceInfo(BaseModel):
    title: str = Field("Title")
    version: str = Field("0.0.1")
    description: str = Field("Description of the service")
    contact: ContactInfo = Field(
        ...,
    )

In [ ]:
# | output: false

my_contact = ContactInfo(
    name="ACME", url="https://www.acme.com", email="noreply@acme.com"
)
service_info = KafkaServiceInfo(contact=my_contact)
pprint(service_info)

KafkaServiceInfo(
│   title='Title',
│   version='0.0.1',
│   description='Description of the service',
│   contact=ContactInfo(name='ACME', url=HttpUrl('https://www.acme.com', ), email='noreply@acme.com')
)

In [ ]:
# | export


class KafkaBrokers(BaseModel):
    brokers: Dict[str, Union[List[KafkaBroker], KafkaBroker]]

    def dict(self, *args: Any, **kwarg: Any) -> Dict[str, Any]:
        """Transcribe brokers into bootstrap server groups"""
        d = super().dict(*args, **kwarg)

        brokers = {}
        for k, v in self.brokers.items():
            if isinstance(v, list):
                brokers.update(
                    {f"{k}-bootstrap-server-{i}": u_v.dict() for i, u_v in enumerate(v)}
                )
            else:
                brokers.update({f"{k}": v.dict()})
        d["brokers"] = brokers
        d = {k: v for k, v in d.items() if v is not None}

        return d

    def json(self, *args: Any, **kwargs: Any) -> str:
        """Serialize into JSON using dict()"""
        return json.dumps(self.dict(), *args, **kwargs)

In [ ]:
# | output: false

kafka_brokers = KafkaBrokers(brokers={"dev": [kafka_broker], "staging": [sec_kafka_broker]})
pprint(kafka_brokers)

expected = '{"brokers": {"dev-bootstrap-server-0": {"url": "kafka", "description": "Kafka broker", "protocol": "kafka", "variables": {"port": {"default": "9092"}}}, "staging-bootstrap-server-0": {"url": "kafka", "description": "Kafka broker", "protocol": "kafka-secure", "security": {"type": "plain"}, "variables": {"port": {"default": "9092"}}}}}'

actual = kafka_brokers.json()
print(f"JSON={actual}")
assert actual == expected, actual

actual = KafkaBrokers.parse_raw(kafka_brokers.json())
pprint(actual)
assert actual == kafka_brokers

KafkaBrokers(
│   brokers={
│   │   'dev': [
│   │   │   KafkaBroker(url='kafka', description='Kafka broker', port='9092', protocol='kafka', security=None)
│   │   ],
│   │   'staging': [
│   │   │   KafkaBroker(
│   │   │   │   url='kafka',
│   │   │   │   description='Kafka broker',
│   │   │   │   port='9092',
│   │   │   │   protocol='kafka-secure',
│   │   │   │   security=SecuritySchema(
│   │   │   │   │   security_type=<SecurityType.plain: 'plain'>,
│   │   │   │   │   description=None,
│   │   │   │   │   name=None,
│   │   │   │   │   api_key_loc=None,
│   │   │   │   │   scheme=None,
│   │   │   │   │   bearerFormat=None,
│   │   │   │   │   flows=None,
│   │   │   │   │   openIdConnectUrl=None
│   │   │   │   )
│   │   │   )
│   │   ]
│   }
)

JSON={"brokers": {"dev-bootstrap-server-0": {"url": "kafka", "description": "Kafka broker", "protocol": "kafka", "variables": {"port": {"default": "9092"}}}, "staging-bootstrap-server-0": {"url": "kafka", "description": "Kafka broker", "protocol": "kafka-secure", "security": {"type": "plain"}, "variables": {"port": {"default": "9092"}}}}}


KafkaBrokers(
│   brokers={
│   │   'dev-bootstrap-server-0': KafkaBroker(
│   │   │   url='kafka',
│   │   │   description='Kafka broker',
│   │   │   port='9092',
│   │   │   protocol='kafka',
│   │   │   security=None
│   │   ),
│   │   'staging-bootstrap-server-0': KafkaBroker(
│   │   │   url='kafka',
│   │   │   description='Kafka broker',
│   │   │   port='9092',
│   │   │   protocol='kafka-secure',
│   │   │   security=SecuritySchema(
│   │   │   │   security_type=<SecurityType.plain: 'plain'>,
│   │   │   │   description=None,
│   │   │   │   name=None,
│   │   │   │   api_key_loc=None,
│   │   │   │   scheme=None,
│   │   │   │   bearerFormat=None,
│   │   │   │   flows=None,
│   │   │   │   openIdConnectUrl=None
│   │   │   )
│   │   )
│   }
)

In [ ]:
# | output: false

brokers_json = '{"brokers": {"dev": {"url": "kafka", "description": "Kafka broker", "protocol": "kafka", "variables": {"port": {"default": "9092"}}}, "staging": {"url": "kafka", "description": "Kafka broker", "protocol": "kafka-secure", "security": {"type": "plain"}, "variables": {"port": {"default": "9092"}}}}}'
kafka_brokers = KafkaBrokers.parse_raw(brokers_json)
pprint(kafka_brokers)

my_contact = ContactInfo(
    name="ACME", url="https://www.acme.com", email="noreply@acme.com"
)
kafka_service_info = KafkaServiceInfo(contact=my_contact)
pprint(kafka_service_info)


class MyInfo(KafkaMessage):
    mobile: str = Field(..., example="+385987654321")
    name: str = Field(..., example="James Bond")


class MyMsgUrl(KafkaMessage):
    info: MyInfo = Field(..., example=dict(mobile="+385987654321", name="James Bond"))
    url: HttpUrl = Field(..., example="https://sis.gov.uk/agents/007")


class MyMsgEmail(KafkaMessage):
    msg_url: MyMsgUrl = Field(
        ...,
        example=dict(
            info=dict(mobile="+385987654321", name="James Bond"),
            url="https://sis.gov.uk/agents/007",
        ),
    )
    email: EmailStr = Field(..., example="agent-007@sis.gov.uk")


def on_my_topic_one(msg: MyMsgUrl) -> None:
    raise NotImplemented


async def on_my_topic_2(msg: MyMsgEmail) -> None:
    raise NotImplemented


async def on_my_topic_2_meta(msg: MyMsgEmail, meta: EventMetadata) -> None:
    raise NotImplemented


async def on_my_topic_2_batch(msg: List[MyMsgEmail]) -> None:
    raise NotImplemented


async def to_my_topic_3(msg) -> MyMsgUrl:
    raise NotImplemented


async def to_my_topic_4(msg) -> KafkaEvent[MyMsgEmail]:
    raise NotImplemented


async def to_my_topic_5(msg) -> KafkaEvent[List[MyMsgEmail]]:
    raise NotImplemented


consumers = {"my_topic_1": on_my_topic_one, "my_topic_2": on_my_topic_2}
producers = {
    "my_topic_3": to_my_topic_3,
    "my_topic_4": to_my_topic_4,
    "my_topic_5": to_my_topic_5,
}

pprint(dict(consumers=consumers, producers=producers))
assert set(consumers.keys()) == set(["my_topic_1", "my_topic_2"])
assert set(producers.keys()) == set(["my_topic_3", "my_topic_4", "my_topic_5"])

KafkaBrokers(
│   brokers={
│   │   'dev': KafkaBroker(url='kafka', description='Kafka broker', port='9092', protocol='kafka', security=None),
│   │   'staging': KafkaBroker(
│   │   │   url='kafka',
│   │   │   description='Kafka broker',
│   │   │   port='9092',
│   │   │   protocol='kafka-secure',
│   │   │   security=SecuritySchema(
│   │   │   │   security_type=<SecurityType.plain: 'plain'>,
│   │   │   │   description=None,
│   │   │   │   name=None,
│   │   │   │   api_key_loc=None,
│   │   │   │   scheme=None,
│   │   │   │   bearerFormat=None,
│   │   │   │   flows=None,
│   │   │   │   openIdConnectUrl=None
│   │   │   )
│   │   )
│   }
)

KafkaServiceInfo(
│   title='Title',
│   version='0.0.1',
│   description='Description of the service',
│   contact=ContactInfo(name='ACME', url=HttpUrl('https://www.acme.com', ), email='noreply@acme.com')
)

{
│   'consumers': {
│   │   'my_topic_1': <function on_my_topic_one at 0x7f5698e8ec00>,
│   │   'my_topic_2': <function on_my_topic_2 at 0x7f5698ed4540>
│   },
│   'producers': {
│   │   'my_topic_3': <function to_my_topic_3 at 0x7f56989e8cc0>,
│   │   'my_topic_4': <function to_my_topic_4 at 0x7f56989e8d60>,
│   │   'my_topic_5': <function to_my_topic_5 at 0x7f56989e8e00>
│   }
}

In [ ]:
# | export

# T = TypeVar("T")


def _get_msg_cls_for_producer(f: ProduceCallable) -> Type[Any]:
    types = get_type_hints(f)
    return_type = types.pop("return", type(None))
    # @app.producer must define a return value
    if return_type == type(None):
        raise ValueError(
            f"Producer function must have a defined return value, got {return_type} as return value"
        )

    return_type = unwrap_from_kafka_event(return_type)
    return_type = unwrap_list_type(return_type)

    if not hasattr(return_type, "json"):
        raise ValueError(f"Producer function return value must have json method")
    return return_type  # type: ignore

In [ ]:
expected = MyMsgUrl
actual = _get_msg_cls_for_producer(to_my_topic_3)
display(actual)
assert actual == expected

__main__.MyMsgUrl

In [ ]:
expected = MyMsgEmail
actual = _get_msg_cls_for_producer(to_my_topic_4)
display(actual)
assert actual == expected

__main__.MyMsgEmail

In [ ]:
expected = MyMsgEmail
actual = _get_msg_cls_for_producer(to_my_topic_5)
display(actual)
assert actual == expected

__main__.MyMsgEmail

In [ ]:
def no_return(i: int):
    pass


with pytest.raises(ValueError) as e:
    _get_msg_cls_for_producer(no_return)

assert e.value.args == (
    "Producer function must have a defined return value, got <class 'NoneType'> as return value",
)

In [ ]:
# | export


def _get_msg_cls_for_consumer(f: ConsumeCallable) -> Type[Any]:
    types = get_type_hints(f)
    return_type = types.pop("return", type(None))
    types_list = list(types.values())
    # @app.consumer does not return a value
    if return_type != type(None):
        raise ValueError(
            f"Consumer function cannot return any value, got {return_type}"
        )
    # @app.consumer first consumer argument must be a msg which is a subclass of BaseModel
    try:
        msg_type = types_list[0]

        msg_type = unwrap_list_type(msg_type)

        if not issubclass(msg_type, BaseModel):
            raise ValueError(
                f"Consumer function first param must be a BaseModel subclass msg, got {types_list}"
            )

        return msg_type  # type: ignore

    except IndexError:
        raise ValueError(
            f"Consumer function first param must be a BaseModel subclass msg, got {types_list}"
        )

In [ ]:
expected = MyMsgUrl
actual = _get_msg_cls_for_consumer(on_my_topic_one)
display(actual)
assert actual == expected

__main__.MyMsgUrl

In [ ]:
expected = MyMsgEmail
actual = _get_msg_cls_for_consumer(on_my_topic_2_meta)
display(actual)
assert actual == expected

__main__.MyMsgEmail

In [ ]:
expected = MyMsgEmail
actual = _get_msg_cls_for_consumer(on_my_topic_2_batch)
display(actual)
assert actual == expected

__main__.MyMsgEmail

In [ ]:
def no_input():
    pass


with pytest.raises(ValueError) as e:
    _get_msg_cls_for_consumer(no_input)

assert e.value.args == (
    "Consumer function first param must be a BaseModel subclass msg, got []",
)


def has_return(a: int) -> int:
    pass


with pytest.raises(ValueError) as e:
    _get_msg_cls_for_consumer(has_return)

assert e.value.args == ("Consumer function cannot return any value, got <class 'int'>",)

In [ ]:
# |export


def _get_topic_dict(
    f: Callable[[Any], Any],
    direction: str = "publish",
) -> Dict[str, Any]:
    if not direction in ["publish", "subscribe"]:
        raise ValueError(
            f"direction must be one of ['publish', 'subscribe'], but it is '{direction}'."
        )

    #     msg_cls = None

    if direction == "publish":
        msg_cls = _get_msg_cls_for_producer(f)
    elif direction == "subscribe":
        msg_cls = _get_msg_cls_for_consumer(f)

    msg_schema = {"message": {"$ref": f"#/components/messages/{msg_cls.__name__}"}}
    if hasattr(f, "description"):
        msg_schema["description"] = getattr(f, "description")
    elif f.__doc__ is not None:
        msg_schema["description"] = f.__doc__  # type: ignore
    return {direction: msg_schema}

In [ ]:
# | output: false

expected = {"subscribe": {"message": {"$ref": "#/components/messages/MyMsgEmail"}}}

actual = _get_topic_dict(on_my_topic_2, "subscribe")
pprint(actual)

assert actual == expected

{'subscribe': {'message': {'$ref': '#/components/messages/MyMsgEmail'}}}

In [ ]:
# | output: false

expected = {"publish": {"message": {"$ref": "#/components/messages/MyMsgEmail"}}}

actual = _get_topic_dict(to_my_topic_4, "publish")
pprint(actual)

assert actual == expected

{'publish': {'message': {'$ref': '#/components/messages/MyMsgEmail'}}}

In [ ]:
# | output: false

expected = {
    "publish": {
        "message": {"$ref": "#/components/messages/MyMsgEmail"},
        "description": "Topic description",
    }
}

setattr(to_my_topic_4, "description", "Topic description")

actual = _get_topic_dict(to_my_topic_4, "publish")
pprint(actual)

assert actual == expected

{'publish': {'message': {'$ref': '#/components/messages/MyMsgEmail'}, 'description': 'Topic description'}}

In [ ]:
# | export


def _get_channels_schema(
    consumers: Dict[str, ConsumeCallable],
    producers: Dict[str, ProduceCallable],
) -> Dict[str, Dict[str, Dict[str, Any]]]:
    topics = {}
    for ms, d in zip([consumers, producers], ["subscribe", "publish"]):
        for topic, f in ms.items():  # type: ignore
            topics[topic] = _get_topic_dict(f, d)
    return topics

In [ ]:
# | output: false

expected = {
    "my_topic_1": {
        "subscribe": {"message": {"$ref": "#/components/messages/MyMsgUrl"}}
    },
    "my_topic_2": {
        "subscribe": {"message": {"$ref": "#/components/messages/MyMsgEmail"}}
    },
    "my_topic_3": {"publish": {"message": {"$ref": "#/components/messages/MyMsgUrl"}}},
    "my_topic_4": {
        "publish": {"message": {"$ref": "#/components/messages/MyMsgEmail"}, 'description': 'Topic description'}
    },
    "my_topic_5": {
        "publish": {"message": {"$ref": "#/components/messages/MyMsgEmail"}}
    },
}
actual = _get_channels_schema(consumers, producers)
pprint(actual)

assert actual == expected

{
│   'my_topic_1': {'subscribe': {'message': {'$ref': '#/components/messages/MyMsgUrl'}}},
│   'my_topic_2': {'subscribe': {'message': {'$ref': '#/components/messages/MyMsgEmail'}}},
│   'my_topic_3': {'publish': {'message': {'$ref': '#/components/messages/MyMsgUrl'}}},
│   'my_topic_4': {
│   │   'publish': {'message': {'$ref': '#/components/messages/MyMsgEmail'}, 'description': 'Topic description'}
│   },
│   'my_topic_5': {'publish': {'message': {'$ref': '#/components/messages/MyMsgEmail'}}}
}

In [ ]:
# | export


def _get_kafka_msg_classes(
    consumers: Dict[str, ConsumeCallable],
    producers: Dict[str, ProduceCallable],
) -> Set[Type[BaseModel]]:
    fc = [_get_msg_cls_for_consumer(consumer) for consumer in consumers.values()]
    fp = [_get_msg_cls_for_producer(producer) for producer in producers.values()]
    return set(fc + fp)


def _get_kafka_msg_definitions(
    consumers: Dict[str, ConsumeCallable],
    producers: Dict[str, ProduceCallable],
) -> Dict[str, Dict[str, Any]]:
    return schema(_get_kafka_msg_classes(consumers, producers))  # type: ignore

In [ ]:
expected = {
    "definitions": {
        "MyInfo": {
            "title": "MyInfo",
            "type": "object",
            "properties": {
                "mobile": {
                    "title": "Mobile",
                    "example": "+385987654321",
                    "type": "string",
                },
                "name": {"title": "Name", "example": "James Bond", "type": "string"},
            },
            "required": ["mobile", "name"],
        },
        "MyMsgUrl": {
            "title": "MyMsgUrl",
            "type": "object",
            "properties": {
                "info": {
                    "title": "Info",
                    "example": {"mobile": "+385987654321", "name": "James Bond"},
                    "allOf": [{"$ref": "#/definitions/MyInfo"}],
                },
                "url": {
                    "title": "Url",
                    "example": "https://sis.gov.uk/agents/007",
                    "minLength": 1,
                    "maxLength": 2083,
                    "format": "uri",
                    "type": "string",
                },
            },
            "required": ["info", "url"],
        },
        "MyMsgEmail": {
            "title": "MyMsgEmail",
            "type": "object",
            "properties": {
                "msg_url": {
                    "title": "Msg Url",
                    "example": {
                        "info": {"mobile": "+385987654321", "name": "James Bond"},
                        "url": "https://sis.gov.uk/agents/007",
                    },
                    "allOf": [{"$ref": "#/definitions/MyMsgUrl"}],
                },
                "email": {
                    "title": "Email",
                    "example": "agent-007@sis.gov.uk",
                    "type": "string",
                    "format": "email",
                },
            },
            "required": ["msg_url", "email"],
        },
    }
}

msg_definitions = _get_kafka_msg_definitions(consumers, producers)
assert msg_definitions == expected

In [ ]:
# | export


def _get_example(cls: Type[BaseModel]) -> BaseModel:
    kwargs: Dict[str, Any] = {}
    for k, v in cls.__fields__.items():
        #         try:
        if (
            hasattr(v, "field_info")
            and hasattr(v.field_info, "extra")
            and "example" in v.field_info.extra
        ):
            example = v.field_info.extra["example"]
            kwargs[k] = example
    #         except:
    #             pass

    return json.loads(cls(**kwargs).json())  # type: ignore

In [ ]:
# | output: false

expected = {
    "msg_url": {
        "info": {"name": "James Bond", "mobile": "+385987654321"},
        "url": "https://sis.gov.uk/agents/007",
    },
    "email": "agent-007@sis.gov.uk",
}

actual = _get_example(MyMsgEmail)
pprint(actual)

assert actual == expected

{
│   'msg_url': {
│   │   'info': {'mobile': '+385987654321', 'name': 'James Bond'},
│   │   'url': 'https://sis.gov.uk/agents/007'
│   },
│   'email': 'agent-007@sis.gov.uk'
}

In [ ]:
# | export


def _add_example_to_msg_definitions(
    msg_cls: Type[BaseModel], msg_schema: Dict[str, Dict[str, Any]]
) -> None:
    try:
        example = _get_example(msg_cls)
    except Exception as e:
        example = None
    if example is not None:
        msg_schema["definitions"][msg_cls.__name__]["example"] = example


def _get_msg_definitions_with_examples(
    consumers: Dict[str, ConsumeCallable],
    producers: Dict[str, ProduceCallable],
) -> Dict[str, Dict[str, Any]]:
    msg_classes = _get_kafka_msg_classes(consumers, producers)
    msg_schema: Dict[str : Dict[str, Any]] = schema(msg_classes)  # type: ignore
    for msg_cls in msg_classes:
        _add_example_to_msg_definitions(msg_cls, msg_schema)

    msg_schema = (
        {k: {"payload": v} for k, v in msg_schema["definitions"].items()}
        if "definitions" in msg_schema
        else {}
    )

    return msg_schema

In [ ]:
# | output: false

expected = {
    "MyInfo": {
        "payload": {
            "title": "MyInfo",
            "type": "object",
            "properties": {
                "mobile": {
                    "title": "Mobile",
                    "example": "+385987654321",
                    "type": "string",
                },
                "name": {"title": "Name", "example": "James Bond", "type": "string"},
            },
            "required": ["mobile", "name"],
        }
    },
    "MyMsgUrl": {
        "payload": {
            "title": "MyMsgUrl",
            "type": "object",
            "properties": {
                "info": {
                    "title": "Info",
                    "example": {"mobile": "+385987654321", "name": "James Bond"},
                    "allOf": [{"$ref": "#/definitions/MyInfo"}],
                },
                "url": {
                    "title": "Url",
                    "example": "https://sis.gov.uk/agents/007",
                    "minLength": 1,
                    "maxLength": 2083,
                    "format": "uri",
                    "type": "string",
                },
            },
            "required": ["info", "url"],
            "example": {
                "info": {"mobile": "+385987654321", "name": "James Bond"},
                "url": "https://sis.gov.uk/agents/007",
            },
        }
    },
    "MyMsgEmail": {
        "payload": {
            "title": "MyMsgEmail",
            "type": "object",
            "properties": {
                "msg_url": {
                    "title": "Msg Url",
                    "example": {
                        "info": {"mobile": "+385987654321", "name": "James Bond"},
                        "url": "https://sis.gov.uk/agents/007",
                    },
                    "allOf": [{"$ref": "#/definitions/MyMsgUrl"}],
                },
                "email": {
                    "title": "Email",
                    "example": "agent-007@sis.gov.uk",
                    "type": "string",
                    "format": "email",
                },
            },
            "required": ["msg_url", "email"],
            "example": {
                "msg_url": {
                    "info": {"mobile": "+385987654321", "name": "James Bond"},
                    "url": "https://sis.gov.uk/agents/007",
                },
                "email": "agent-007@sis.gov.uk",
            },
        }
    },
}

actual = _get_msg_definitions_with_examples(consumers, producers)
pprint(actual)
assert actual == expected

{
│   'MyInfo': {
│   │   'payload': {
│   │   │   'title': 'MyInfo',
│   │   │   'type': 'object',
│   │   │   'properties': {
│   │   │   │   'mobile': {'title': 'Mobile', 'example': '+385987654321', 'type': 'string'},
│   │   │   │   'name': {'title': 'Name', 'example': 'James Bond', 'type': 'string'}
│   │   │   },
│   │   │   'required': ['mobile', 'name']
│   │   }
│   },
│   'MyMsgUrl': {
│   │   'payload': {
│   │   │   'title': 'MyMsgUrl',
│   │   │   'type': 'object',
│   │   │   'properties': {
│   │   │   │   'info': {
│   │   │   │   │   'title': 'Info',
│   │   │   │   │   'example': {'mobile': '+385987654321', 'name': 'James Bond'},
│   │   │   │   │   'allOf': [{'$ref': '#/definitions/MyInfo'}]
│   │   │   │   },
│   │   │   │   'url': {
│   │   │   │   │   'title': 'Url',
│   │   │   │   │   'example': 'https://sis.gov.uk/agents/007',
│   │   │   │   │   'minLength': 1,
│   │   │   │   │   'maxLength': 2083,
│   │   │   │   │   'format': 'uri',
│   │   │   │   │   'type': 'string'
│   │   │   │   }
│   │   │   },
│   │   │   'required': ['info', 'url'],
│   │   │   'example': {
│   │   │   │   'info': {'mobile': '+385987654321', 'name': 'James Bond'},
│   │   │   │   'url': 'https://sis.gov.uk/agents/007'
│   │   │   }
│   │   }
│   },
│   'MyMsgEmail': {
│   │   'payload': {
│   │   │   'title': 'MyMsgEmail',
│   │   │   'type': 'object',
│   │   │   'properties': {
│   │   │   │   'msg_url': {
│   │   │   │   │   'title': 'Msg Url',
│   │   │   │   │   'example': {
│   │   │   │   │   │   'info': {'mobile': '+385987654321', 'name': 'James Bond'},
│   │   │   │   │   │   'url': 'https://sis.gov.uk/agents/007'
│   │   │   │   │   },
│   │   │   │   │   'allOf': [{'$ref': '#/definitions/MyMsgUrl'}]
│   │   │   │   },
│   │   │   │   'email': {
│   │   │   │   │   'title': 'Email',
│   │   │   │   │   'example': 'agent-007@sis.gov.uk',
│   │   │   │   │   'type': 'string',
│   │   │   │   │   'format': 'email'
│   │   │   │   }
│   │   │   },
│   │   │   'required': ['msg_url', 'email'],
│   │   │   'example': {
│   │   │   │   'msg_url': {
│   │   │   │   │   'info': {'mobile': '+385987654321', 'name': 'James Bond'},
│   │   │   │   │   'url': 'https://sis.gov.uk/agents/007'
│   │   │   │   },
│   │   │   │   'email': 'agent-007@sis.gov.uk'
│   │   │   }
│   │   }
│   }
}

In [ ]:
# | export


def _get_security_schemes(kafka_brokers: KafkaBrokers) -> Dict[str, Any]:
    security_schemes = {}
    for key, broker in kafka_brokers.brokers.items():
        if isinstance(broker, list):
            kafka_broker = broker[0]
        else:
            kafka_broker = broker

        if kafka_broker.security is not None:
            security_schemes[f"{key}_default_security"] = json.loads(
                kafka_broker.security.json()
            )
    return security_schemes

In [ ]:
# | export


def _get_components_schema(
    consumers: Dict[str, ConsumeCallable],
    producers: Dict[str, ProduceCallable],
    kafka_brokers: KafkaBrokers,
) -> Dict[str, Any]:
    definitions = _get_msg_definitions_with_examples(consumers, producers)
    msg_classes = [cls.__name__ for cls in _get_kafka_msg_classes(consumers, producers)]
    components = {
        "messages": {k: v for k, v in definitions.items() if k in msg_classes},
        "schemas": {k: v for k, v in definitions.items() if k not in msg_classes},
        "securitySchemes": _get_security_schemes(kafka_brokers),
    }
    substitutions = {
        f"#/definitions/{k}": f"#/components/messages/{k}"
        if k in msg_classes
        else f"#/components/schemas/{k}"
        for k in definitions.keys()
    }

    def _sub_values(d: Any, substitutions: Dict[str, str] = substitutions) -> Any:
        if isinstance(d, dict):
            d = {k: _sub_values(v) for k, v in d.items()}
        if isinstance(d, list):
            d = [_sub_values(k) for k in d]
        elif isinstance(d, str):
            for k, v in substitutions.items():
                if d == k:
                    d = v
        return d

    return _sub_values(components)  # type: ignore

In [ ]:
# | output: false

components = _get_components_schema(consumers, producers, kafka_brokers)
pprint(components)

{
│   'messages': {
│   │   'MyMsgUrl': {
│   │   │   'payload': {
│   │   │   │   'title': 'MyMsgUrl',
│   │   │   │   'type': 'object',
│   │   │   │   'properties': {
│   │   │   │   │   'info': {
│   │   │   │   │   │   'title': 'Info',
│   │   │   │   │   │   'example': {'mobile': '+385987654321', 'name': 'James Bond'},
│   │   │   │   │   │   'allOf': [{'$ref': '#/components/schemas/MyInfo'}]
│   │   │   │   │   },
│   │   │   │   │   'url': {
│   │   │   │   │   │   'title': 'Url',
│   │   │   │   │   │   'example': 'https://sis.gov.uk/agents/007',
│   │   │   │   │   │   'minLength': 1,
│   │   │   │   │   │   'maxLength': 2083,
│   │   │   │   │   │   'format': 'uri',
│   │   │   │   │   │   'type': 'string'
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   'required': ['info', 'url'],
│   │   │   │   'example': {
│   │   │   │   │   'info': {'mobile': '+385987654321', 'name': 'James Bond'},
│   │   │   │   │   'url': 'https://sis.gov.uk/agents/007'
│   │   │   │   }
│   │   │   }
│   │   },
│   │   'MyMsgEmail': {
│   │   │   'payload': {
│   │   │   │   'title': 'MyMsgEmail',
│   │   │   │   'type': 'object',
│   │   │   │   'properties': {
│   │   │   │   │   'msg_url': {
│   │   │   │   │   │   'title': 'Msg Url',
│   │   │   │   │   │   'example': {
│   │   │   │   │   │   │   'info': {'mobile': '+385987654321', 'name': 'James Bond'},
│   │   │   │   │   │   │   'url': 'https://sis.gov.uk/agents/007'
│   │   │   │   │   │   },
│   │   │   │   │   │   'allOf': [{'$ref': '#/components/messages/MyMsgUrl'}]
│   │   │   │   │   },
│   │   │   │   │   'email': {
│   │   │   │   │   │   'title': 'Email',
│   │   │   │   │   │   'example': 'agent-007@sis.gov.uk',
│   │   │   │   │   │   'type': 'string',
│   │   │   │   │   │   'format': 'email'
│   │   │   │   │   }
│   │   │   │   },
│   │   │   │   'required': ['msg_url', 'email'],
│   │   │   │   'example': {
│   │   │   │   │   'msg_url': {
│   │   │   │   │   │   'info': {'mobile': '+385987654321', 'name': 'James Bond'},
│   │   │   │   │   │   'url': 'https://sis.gov.uk/agents/007'
│   │   │   │   │   },
│   │   │   │   │   'email': 'agent-007@sis.gov.uk'
│   │   │   │   }
│   │   │   }
│   │   }
│   },
│   'schemas': {
│   │   'MyInfo': {
│   │   │   'payload': {
│   │   │   │   'title': 'MyInfo',
│   │   │   │   'type': 'object',
│   │   │   │   'properties': {
│   │   │   │   │   'mobile': {'title': 'Mobile', 'example': '+385987654321', 'type': 'string'},
│   │   │   │   │   'name': {'title': 'Name', 'example': 'James Bond', 'type': 'string'}
│   │   │   │   },
│   │   │   │   'required': ['mobile', 'name']
│   │   │   }
│   │   }
│   },
│   'securitySchemes': {'staging_default_security': {'type': 'plain'}}
}

In [ ]:
# | export


def _get_servers_schema(kafka_brokers: KafkaBrokers) -> Dict[str, Any]:
    servers = json.loads(kafka_brokers.json(sort_keys=False))["brokers"]

    for key, kafka_broker in servers.items():
        if "security" in kafka_broker:
            servers[key]["security"] = [{f"{key}_default_security": []}]
    return servers  # type: ignore

In [ ]:
# | output: false

expected = {
    "dev": {
        "url": "kafka",
        "description": "Kafka broker",
        "protocol": "kafka",
        "variables": {"port": {"default": "9092"}},
    },
    "staging": {
        "url": "kafka",
        "description": "Kafka broker",
        "protocol": "kafka-secure",
        "security": [{"staging_default_security": []}],
        "variables": {"port": {"default": "9092"}},
    },
}

actual = _get_servers_schema(kafka_brokers)
pprint(actual)
assert actual == expected, actual

{
│   'dev': {
│   │   'url': 'kafka',
│   │   'description': 'Kafka broker',
│   │   'protocol': 'kafka',
│   │   'variables': {'port': {'default': '9092'}}
│   },
│   'staging': {
│   │   'url': 'kafka',
│   │   'description': 'Kafka broker',
│   │   'protocol': 'kafka-secure',
│   │   'security': [{'staging_default_security': []}],
│   │   'variables': {'port': {'default': '9092'}}
│   }
}

In [ ]:
# | export


def _get_asyncapi_schema(
    consumers: Dict[str, ConsumeCallable],
    producers: Dict[str, ProduceCallable],
    kafka_brokers: KafkaBrokers,
    kafka_service_info: KafkaServiceInfo,
) -> Dict[str, Any]:
    #     # we don't use dict because we need custom JSON encoders
    info = json.loads(kafka_service_info.json(sort_keys=False))
    servers = _get_servers_schema(kafka_brokers)
    #     # should be in the proper format already
    channels = _get_channels_schema(consumers, producers)
    components = _get_components_schema(consumers, producers, kafka_brokers)
    return {
        "asyncapi": "2.5.0",
        "info": info,
        "servers": servers,
        "channels": channels,
        "components": components,
    }

In [ ]:
# | output: false

expected = {
    "asyncapi": "2.5.0",
    "info": {
        "title": "Title",
        "version": "0.0.1",
        "description": "Description of the service",
        "contact": {
            "name": "ACME",
            "url": "https://www.acme.com",
            "email": "noreply@acme.com",
        },
    },
    "servers": {
        "dev": {
            "url": "kafka",
            "description": "Kafka broker",
            "protocol": "kafka",
            "variables": {"port": {"default": "9092"}},
        },
        "staging": {
            "url": "kafka",
            "description": "Kafka broker",
            "protocol": "kafka-secure",
            "security": [{"staging_default_security": []}],
            "variables": {"port": {"default": "9092"}},
        },
    },
    "channels": {
        "my_topic_1": {
            "subscribe": {"message": {"$ref": "#/components/messages/MyMsgUrl"}}
        },
        "my_topic_2": {
            "subscribe": {"message": {"$ref": "#/components/messages/MyMsgEmail"}}
        },
        "my_topic_3": {
            "publish": {"message": {"$ref": "#/components/messages/MyMsgUrl"}}
        },
        "my_topic_4": {
            "publish": {
                "message": {"$ref": "#/components/messages/MyMsgEmail"},
                "description": "Topic description",
            }
        },
        "my_topic_5": {
            "publish": {"message": {"$ref": "#/components/messages/MyMsgEmail"}}
        },
    },
    "components": {
        "securitySchemes": {"staging_default_security": {"type": "plain"}},
        "messages": {
            "MyMsgUrl": {
                "payload": {
                    "title": "MyMsgUrl",
                    "type": "object",
                    "properties": {
                        "info": {
                            "title": "Info",
                            "example": {
                                "mobile": "+385987654321",
                                "name": "James Bond",
                            },
                            "allOf": [{"$ref": "#/components/schemas/MyInfo"}],
                        },
                        "url": {
                            "title": "Url",
                            "example": "https://sis.gov.uk/agents/007",
                            "minLength": 1,
                            "maxLength": 2083,
                            "format": "uri",
                            "type": "string",
                        },
                    },
                    "required": ["info", "url"],
                    "example": {
                        "info": {"mobile": "+385987654321", "name": "James Bond"},
                        "url": "https://sis.gov.uk/agents/007",
                    },
                }
            },
            "MyMsgEmail": {
                "payload": {
                    "title": "MyMsgEmail",
                    "type": "object",
                    "properties": {
                        "msg_url": {
                            "title": "Msg Url",
                            "example": {
                                "info": {
                                    "mobile": "+385987654321",
                                    "name": "James Bond",
                                },
                                "url": "https://sis.gov.uk/agents/007",
                            },
                            "allOf": [{"$ref": "#/components/messages/MyMsgUrl"}],
                        },
                        "email": {
                            "title": "Email",
                            "example": "agent-007@sis.gov.uk",
                            "type": "string",
                            "format": "email",
                        },
                    },
                    "required": ["msg_url", "email"],
                    "example": {
                        "msg_url": {
                            "info": {"mobile": "+385987654321", "name": "James Bond"},
                            "url": "https://sis.gov.uk/agents/007",
                        },
                        "email": "agent-007@sis.gov.uk",
                    },
                }
            },
        },
        "schemas": {
            "MyInfo": {
                "payload": {
                    "title": "MyInfo",
                    "type": "object",
                    "properties": {
                        "mobile": {
                            "title": "Mobile",
                            "example": "+385987654321",
                            "type": "string",
                        },
                        "name": {
                            "title": "Name",
                            "example": "James Bond",
                            "type": "string",
                        },
                    },
                    "required": ["mobile", "name"],
                }
            }
        },
    },
}
asyncapi_schema = _get_asyncapi_schema(
    consumers, producers, kafka_brokers, kafka_service_info
)
pprint(asyncapi_schema)
assert asyncapi_schema == expected

{
│   'asyncapi': '2.5.0',
│   'info': {
│   │   'title': 'Title',
│   │   'version': '0.0.1',
│   │   'description': 'Description of the service',
│   │   'contact': {'name': 'ACME', 'url': 'https://www.acme.com', 'email': 'noreply@acme.com'}
│   },
│   'servers': {
│   │   'dev': {
│   │   │   'url': 'kafka',
│   │   │   'description': 'Kafka broker',
│   │   │   'protocol': 'kafka',
│   │   │   'variables': {'port': {'default': '9092'}}
│   │   },
│   │   'staging': {
│   │   │   'url': 'kafka',
│   │   │   'description': 'Kafka broker',
│   │   │   'protocol': 'kafka-secure',
│   │   │   'security': [{'staging_default_security': []}],
│   │   │   'variables': {'port': {'default': '9092'}}
│   │   }
│   },
│   'channels': {
│   │   'my_topic_1': {'subscribe': {'message': {'$ref': '#/components/messages/MyMsgUrl'}}},
│   │   'my_topic_2': {'subscribe': {'message': {'$ref': '#/components/messages/MyMsgEmail'}}},
│   │   'my_topic_3': {'publish': {'message': {'$ref': '#/components/messages/MyMsgUrl'}}},
│   │   'my_topic_4': {
│   │   │   'publish': {
│   │   │   │   'message': {'$ref': '#/components/messages/MyMsgEmail'},
│   │   │   │   'description': 'Topic description'
│   │   │   }
│   │   },
│   │   'my_topic_5': {'publish': {'message': {'$ref': '#/components/messages/MyMsgEmail'}}}
│   },
│   'components': {
│   │   'messages': {
│   │   │   'MyMsgUrl': {
│   │   │   │   'payload': {
│   │   │   │   │   'title': 'MyMsgUrl',
│   │   │   │   │   'type': 'object',
│   │   │   │   │   'properties': {
│   │   │   │   │   │   'info': {
│   │   │   │   │   │   │   'title': 'Info',
│   │   │   │   │   │   │   'example': {'mobile': '+385987654321', 'name': 'James Bond'},
│   │   │   │   │   │   │   'allOf': [{'$ref': '#/components/schemas/MyInfo'}]
│   │   │   │   │   │   },
│   │   │   │   │   │   'url': {
│   │   │   │   │   │   │   'title': 'Url',
│   │   │   │   │   │   │   'example': 'https://sis.gov.uk/agents/007',
│   │   │   │   │   │   │   'minLength': 1,
│   │   │   │   │   │   │   'maxLength': 2083,
│   │   │   │   │   │   │   'format': 'uri',
│   │   │   │   │   │   │   'type': 'string'
│   │   │   │   │   │   }
│   │   │   │   │   },
│   │   │   │   │   'required': ['info', 'url'],
│   │   │   │   │   'example': {
│   │   │   │   │   │   'info': {'mobile': '+385987654321', 'name': 'James Bond'},
│   │   │   │   │   │   'url': 'https://sis.gov.uk/agents/007'
│   │   │   │   │   }
│   │   │   │   }
│   │   │   },
│   │   │   'MyMsgEmail': {
│   │   │   │   'payload': {
│   │   │   │   │   'title': 'MyMsgEmail',
│   │   │   │   │   'type': 'object',
│   │   │   │   │   'properties': {
│   │   │   │   │   │   'msg_url': {
│   │   │   │   │   │   │   'title': 'Msg Url',
│   │   │   │   │   │   │   'example': {
│   │   │   │   │   │   │   │   'info': {'mobile': '+385987654321', 'name': 'James Bond'},
│   │   │   │   │   │   │   │   'url': 'https://sis.gov.uk/agents/007'
│   │   │   │   │   │   │   },
│   │   │   │   │   │   │   'allOf': [{'$ref': '#/components/messages/MyMsgUrl'}]
│   │   │   │   │   │   },
│   │   │   │   │   │   'email': {
│   │   │   │   │   │   │   'title': 'Email',
│   │   │   │   │   │   │   'example': 'agent-007@sis.gov.uk',
│   │   │   │   │   │   │   'type': 'string',
│   │   │   │   │   │   │   'format': 'email'
│   │   │   │   │   │   }
│   │   │   │   │   },
│   │   │   │   │   'required': ['msg_url', 'email'],
│   │   │   │   │   'example': {
│   │   │   │   │   │   'msg_url': {
│   │   │   │   │   │   │   'info': {'mobile': '+385987654321', 'name': 'James Bond'},
│   │   │   │   │   │   │   'url': 'https://sis.gov.uk/agents/007'
│   │   │   │   │   │   },
│   │   │   │   │   │   'email': 'agent-007@sis.gov.uk'
│   │   │   │   │   }
│   │   │   │   }
│   │   │   }
│   │   },
│   │   'schemas': {
│   │   │   'MyInfo': {
│   │   │   │   'payload': {
│   │   │   │   │   'title': 'MyInfo',
│   │   │   │   │   'type': 'object',
│   │   │   │   │   'properties': {
│   │   │   │   │   │   'mobile': 

In [ ]:
# |export


def yaml_file_cmp(file_1: Union[Path, str], file_2: Union[Path, str]) -> bool:
    """Compares two YAML files and returns True if their contents are equal, False otherwise.

    Args:
        file_1: Path or string representing the first YAML file.
        file_2: Path or string representing the second YAML file.

    Returns:
        A boolean indicating whether the contents of the two YAML files are equal.
    """
    try:
        import yaml
    except Exception as e:
        msg = "Please install docs version of fastkafka using 'pip install fastkafka[docs]' command"
        logger.error(msg)
        raise RuntimeError(msg)

    def _read(f: Union[Path, str]) -> Dict[str, Any]:
        with open(f) as stream:
            return yaml.safe_load(stream)  # type: ignore

    d = [_read(f) for f in [file_1, file_2]]
    return d[0] == d[1]

In [ ]:
# | export


def _generate_async_spec(
    *,
    consumers: Dict[str, ConsumeCallable],
    producers: Dict[str, ProduceCallable],
    kafka_brokers: KafkaBrokers,
    kafka_service_info: KafkaServiceInfo,
    spec_path: Path,
    force_rebuild: bool,
) -> bool:
    try:
        import yaml
    except Exception as e:
        msg = "Please install docs version of fastkafka using 'pip install fastkafka[docs]' command"
        logger.error(msg)
        raise RuntimeError(msg)

    # generate spec file
    asyncapi_schema = _get_asyncapi_schema(
        consumers, producers, kafka_brokers, kafka_service_info
    )
    if not spec_path.exists():
        logger.info(
            f"Old async specifications at '{spec_path.resolve()}' does not exist."
        )
    spec_path.parent.mkdir(exist_ok=True, parents=True)
    with tempfile.TemporaryDirectory() as d:
        with open(Path(d) / "asyncapi.yml", "w") as f:
            yaml.dump(asyncapi_schema, f, sort_keys=False)
        spec_changed = not (
            spec_path.exists() and yaml_file_cmp(Path(d) / "asyncapi.yml", spec_path)
        )
        if spec_changed or force_rebuild:
            shutil.copyfile(Path(d) / "asyncapi.yml", spec_path)
            logger.info(
                f"New async specifications generated at: '{spec_path.resolve()}'"
            )
            return True
        else:
            logger.info(
                f"Keeping the old async specifications at: '{spec_path.resolve()}'"
            )
            return False

In [ ]:
with tempfile.TemporaryDirectory() as d:
    try:
        asyncapi_path = Path(d).parent / "003_AsyncAPI" / "asyncapi"
        if asyncapi_path.exists():
            shutil.rmtree(asyncapi_path)
        spec_path = Path(asyncapi_path) / "spec" / "asyncapi.yml"

        is_spec_built = _generate_async_spec(
            consumers=consumers,
            producers=producers,
            kafka_brokers=kafka_brokers,
            kafka_service_info=kafka_service_info,
            spec_path=spec_path,
            force_rebuild=False,
        )
        assert is_spec_built
        assert (Path(asyncapi_path) / "spec" / "asyncapi.yml").exists()

        is_spec_built = _generate_async_spec(
            consumers=consumers,
            producers=producers,
            kafka_brokers=kafka_brokers,
            kafka_service_info=kafka_service_info,
            spec_path=spec_path,
            force_rebuild=False,
        )
        assert not is_spec_built
        assert (Path(asyncapi_path) / "spec" / "asyncapi.yml").exists()

        is_spec_built = _generate_async_spec(
            consumers=consumers,
            producers=producers,
            kafka_brokers=kafka_brokers,
            kafka_service_info=kafka_service_info,
            spec_path=spec_path,
            force_rebuild=True,
        )
        assert is_spec_built
        assert (Path(asyncapi_path) / "spec" / "asyncapi.yml").exists()

    finally:
        shutil.rmtree(asyncapi_path)

[INFO] __main__: Old async specifications at '/tmp/003_AsyncAPI/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] __main__: New async specifications generated at: '/tmp/003_AsyncAPI/asyncapi/spec/asyncapi.yml'
[INFO] __main__: Keeping the old async specifications at: '/tmp/003_AsyncAPI/asyncapi/spec/asyncapi.yml'
[INFO] __main__: New async specifications generated at: '/tmp/003_AsyncAPI/asyncapi/spec/asyncapi.yml'


In [ ]:
# | export


def _generate_async_docs(
    *,
    spec_path: Path,
    docs_path: Path,
) -> None:
    _check_npm_with_local()
    cmd = [
        "npx",
        "-y",
        "-p",
        "@asyncapi/generator",
        "ag",
        f"{spec_path}",
        "@asyncapi/html-template",
        "-o",
        f"{docs_path}",
        "--force-write",
    ]
    # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
    p = subprocess.run(  # nosec: B602, B603 subprocess call - check for execution of untrusted input.
        cmd,
        stderr=subprocess.STDOUT,
        stdout=subprocess.PIPE,
        shell=True if platform.system() == "Windows" else False,
    )
    if p.returncode == 0:
        logger.info(f"Async docs generated at '{docs_path}'")
        logger.info(f"Output of '$ {' '.join(cmd)}'{p.stdout.decode()}")
    else:
        logger.error(f"Generation of async docs failed!")
        logger.info(f"Output of '$ {' '.join(cmd)}'{p.stdout.decode()}")
        raise ValueError(
            f"Generation of async docs failed, used '$ {' '.join(cmd)}'{p.stdout.decode()}"
        )

In [ ]:
with tempfile.TemporaryDirectory() as d:
    try:
        asyncapi_path = Path(d).parent / "003_AsyncAPI" / "asyncapi"
        if asyncapi_path.exists():
            shutil.rmtree(asyncapi_path)
        spec_path = Path(asyncapi_path) / "spec" / "asyncapi.yml"
        docs_path = Path(asyncapi_path) / "docs"

        is_spec_built = _generate_async_spec(
            consumers=consumers,
            producers=producers,
            kafka_brokers=kafka_brokers,
            kafka_service_info=kafka_service_info,
            spec_path=spec_path,
            force_rebuild=False,
        )

        _generate_async_docs(
            spec_path=spec_path,
            docs_path=docs_path,
        )
        assert docs_path.exists()
    finally:
        shutil.rmtree(asyncapi_path)

[INFO] __main__: Old async specifications at '/tmp/003_AsyncAPI/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] __main__: New async specifications generated at: '/tmp/003_AsyncAPI/asyncapi/spec/asyncapi.yml'
[INFO] __main__: Async docs generated at '/tmp/003_AsyncAPI/asyncapi/docs'
[INFO] __main__: Output of '$ npx -y -p @asyncapi/generator ag /tmp/003_AsyncAPI/asyncapi/spec/asyncapi.yml @asyncapi/html-template -o /tmp/003_AsyncAPI/asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /tmp/003_AsyncAPI/asyncapi/docs.




In [ ]:
# |export


def export_async_spec(
    *,
    consumers: Dict[str, ConsumeCallable],
    producers: Dict[str, ProduceCallable],
    kafka_brokers: KafkaBrokers,
    kafka_service_info: KafkaServiceInfo,
    asyncapi_path: Union[Path, str],
    force_rebuild: bool = True,
) -> None:
    """Exports the AsyncAPI specification and documentation to the given path.

    Args:
        consumers: Dictionary of consumer functions, where the keys are the channel names and the values are the consumer functions.
        producers: Dictionary of producer functions, where the keys are the channel names and the values are the producer functions.
        kafka_brokers: KafkaBrokers object representing the Kafka brokers configuration.
        kafka_service_info: KafkaServiceInfo object representing the Kafka service info configuration.
        asyncapi_path: Path or string representing the base path where the specification and documentation will be exported.
        force_rebuild: Boolean indicating whether to force a rebuild of the specification file even if it already exists.
    """
    # generate spec file
    spec_path = Path(asyncapi_path) / "spec" / "asyncapi.yml"
    is_spec_built = _generate_async_spec(
        consumers=consumers,
        producers=producers,
        kafka_brokers=kafka_brokers,
        kafka_service_info=kafka_service_info,
        spec_path=spec_path,
        force_rebuild=force_rebuild,
    )

    # generate docs folder
    docs_path = Path(asyncapi_path) / "docs"

    if not is_spec_built and docs_path.exists():
        logger.info(
            f"Skipping generating async documentation in '{docs_path.resolve()}'"
        )
        return

    _generate_async_docs(
        spec_path=spec_path,
        docs_path=docs_path,
    )

In [ ]:
with tempfile.TemporaryDirectory() as d:
    try:
        asyncapi_path = Path(d).parent / "003_AsyncAPI" / "asyncapi"
        if asyncapi_path.exists():
            shutil.rmtree(asyncapi_path)

        export_async_spec(
            consumers=consumers,
            producers=producers,
            kafka_brokers=kafka_brokers,
            kafka_service_info=kafka_service_info,
            asyncapi_path=asyncapi_path,
            force_rebuild=False,
        )
        #         !ls -al {asyncapi_path}
        assert (Path(asyncapi_path) / "spec" / "asyncapi.yml").exists()
        assert (Path(asyncapi_path) / "docs" / "index.html").exists()
    finally:
        shutil.rmtree(asyncapi_path)

[INFO] __main__: Old async specifications at '/tmp/003_AsyncAPI/asyncapi/spec/asyncapi.yml' does not exist.
[INFO] __main__: New async specifications generated at: '/tmp/003_AsyncAPI/asyncapi/spec/asyncapi.yml'
[INFO] __main__: Async docs generated at '/tmp/003_AsyncAPI/asyncapi/docs'
[INFO] __main__: Output of '$ npx -y -p @asyncapi/generator ag /tmp/003_AsyncAPI/asyncapi/spec/asyncapi.yml @asyncapi/html-template -o /tmp/003_AsyncAPI/asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /tmp/003_AsyncAPI/asyncapi/docs.


